In [4]:
import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt
import theano.tensor as tt

import time

import _pickle as cPickle


from cantilever import Cantilever

In [5]:
# Set the number of unknown parameters (i.e. dimension of theta in posterior)
nparam = 3

# Number of draws from the distribution
ndraws = 1000

# Number of burn-in samples
nburn = 500

# Metropolis tuning parameters
tune = True
tune_interval = 100
discard_tuning = True

# Number of independent chains
nchains = 4

# Do blocked/compounds sampling in Metropolis and MLDA 
# Note: This choice applies only to the coarsest level in MLDA 
# (where a Metropolis sampler is used), all other levels use block sampling
blocked = True

# Set the sigma for inference
sigma_f = 0.1

# Sampling seed
sampling_seed = 12345




In [8]:
# Parameters

param = [ None ] * 11

param[0] = 4.5  # E_R   GPa
param[1] = 0.35 # nu_R

param[2] = 135  # E1    GPa
param[3] = 8.5  # E2    GPa
param[4] = 8.5  # E3    GPa

param[5] = 0.022    # nu_21
param[6] = 0.022    # nu_31
param[7] = 0.45     # nu_32

param[8] = 5.0  # G_12 GPa
param[9] = 5.0;   # G_13 GPa
param[10] = 5.0;  # G_23 GPa

# Initialised Macro scale model

myModel = Cantilever(param)



In [10]:
# Use a Theano Op along with the code within ./mlda to construct the likelihood

class LogLike(tt.Op):
    itypes = [tt.dvector]  # expects a vector of parameter values when called
    otypes = [tt.dscalar]  # outputs a single scalar value (the log likelihood)

    def __init__(self, my_model, baseAngle, data, sigma_f):

        # add inputs as class attributes
        self.my_model = my_model
        self.baseAngle = baseAngle
        self.data = data
        self.sigma_f = sigma_f
        
    def perform(self, node, inputs, outputs):
        
        theta = inputs  # this will contain my variables
        
        # call the log-likelihood function
        F = self.my_model.solve(self.baseAngle + theta[0])
        
        logl = -0.5 * np.sum((F - self.data)**2) / (self.sigma_f**2)
        
        outputs[0][0] = np.array(logl) # output the log-likelihood

In [11]:
# create Theano Ops to wrap likelihood
logl = LogLike(myModel)

In [ ]:
# Set up finest model and perform inference with PyMC3, using the MLDA algorithm
# and passing the coarse_models list created above.

traces = []
runtimes = []
acc = []
ess = []


with pm.Model():
    
    # Define priors on parameters
    param = []
    
    BoundedNormal = pm.Bound(pm.Normal, lower=0.0)
    
    param.append(BoundedNormal('C10', mu = 0.25, sigma = 1.0))
    param.append(BoundedNormal('C01', mu = 0.25, sigma = 1.0))
    param.append(BoundedNormal('C11', mu = 0.25, sigma = 1.0))
    param.append(BoundedNormal('D1', mu = 0.25, sigma = 1.0))

    # Convert m and c to a tensor vector
    theta = tt.as_tensor_variable(param)

    # use a DensityDist (use a lamdba function to "call" the Op)
    pm.DensityDist('likelihood', lambda v: logl(v), observed={'v': theta})
    
    #map_estimate = pm.find_MAP()

    
    # Initialise a demetropolis step method.
    step_demetropolis = pm.DEMetropolis(tune='scaling', tune_interval=tune_interval)

    # Inference! 
    # DEMetropolis
    t_start = time.time()
    traces.append(pm.sample(draws=ndraws, step=step_demetropolis,
                            chains=nchains, tune=nburn,
                            discard_tuned_samples=discard_tuning,
                            random_seed=sampling_seed))
    runtimes.append(time.time() - t_start)
    

Population sampling (4 chains)
/Users/td336/miniconda3/envs/fenicsproject/lib/python3.8/site-packages/pymc3-3.8-py3.8.egg/pymc3/sampling.py:516: UserWarning: DEMetropolis should be used with more chains than dimensions! (The model has 4 dimensions.)
DEMetropolis: [D1, C11, C01, C10]
Attempting to parallelize chains to all cores. You can turn this off with `pm.sample(cores=1)`.


Population parallelization failed. Falling back to sequential stepping of chains.


Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FF

In [6]:
print(map_estimate)

{'C10_lowerbound__': array(-0.65113079), 'C01_lowerbound__': array(-69.74265335), 'C11_lowerbound__': array(-34.88823224), 'D1_lowerbound__': array(-72.33058231), 'C10': array(0.52145578), 'C01': array(5.14221814e-31), 'C11': array(7.05071653e-16), 'D1': array(3.86569182e-32)}
